In [8]:
import tensorflow as tf
import numpy as np 
from keras import backend as K
from keras import layers
from efficientnet.keras import EfficientNetB3
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow_model_optimization.quantization.keras import vitis_quantize
HEIGHT = 112
WIDTH = 112

efficientnetb3 = EfficientNetB3(
        weights=None,
        input_shape=(HEIGHT,WIDTH,3),
        include_top=False
                   )


def build_model():
    model = Sequential()
    model.add(efficientnetb3)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(5, activation='sigmoid'))
    
#     model.compile(
#         loss='binary_crossentropy',
#         optimizer=Adam(lr=1e-4,decay=1e-6),
#         metrics=['accuracy']
#     )
    
    return model

shape_range =range(32,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    efficientnetb3 = EfficientNetB3(
        weights=None,
        input_shape=(shape_x,shape_y,1),
        include_top=False
                   )
    step=1

    model = build_model()
    model.summary()
    model.save('./EfficientNetB3_h5/EfficientNetB3_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
            --model ./quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel_EfficientNetB3 \
            --net_name deploy
    os.rename("./xmodel_EfficientNetB3/deploy.xmodel","./xmodel_EfficientNetB3/xmodel_EfficientNetB3_{shape_x}_{shape_y}_{step}.xmodel".format(shape_x=shape_x,shape_y=shape_y,step=step))

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b3 (Functional) (None, 1, 1, 1536)        10782808  
_________________________________________________________________
global_average_pooling2d_13  (None, 1536)              0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 1536)              0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 1536)              6144      
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 7685      
Total params: 10,796,637
Trainable params: 10,706,269
Non-trainable params: 90,368
_________________________________________________________________
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_b

ValueError: Cannot assign to variable stem_conv/kernel:0 due to variable shape (3, 3, 1, 40) and value shape (1, 1, 384, 1536) are incompatible